# Project Milestone - ETL Process
---

### Step 0: Install the required python packages

In [ ]:
pip install --upgrade sodapy

In [ ]:
pip install --upgrade db-dtypes

In [ ]:
pip install --upgrade pyarrow

In [ ]:
pip install --upgrade google-cloud-bigquery

### Step 1: Setup your NYC Open Data variables 

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sodapy import Socrata
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
# setup the host name for the API endpoint
data_url = 'data.cityofnewyork.us'

In [3]:
# setup the data set at the API endpoint (Collisions Dataset)
dataset_collisions = 'h9gi-nx95'

In [4]:
# Setup your App Token
app_token = r'y1POUYL9H17SRIrTJjY0I1TbQ'

In [5]:
# create the client that points to the API endpoint
nyc_open_data_client = Socrata(data_url, app_token, timeout = 200)
print(f"nyc open data client name is: {nyc_open_data_client}")
print(f"nyc open data client data type is: {type(nyc_open_data_client)}")

nyc open data client name is: <sodapy.socrata.Socrata object at 0x7fa8dc3a6520>
nyc open data client data type is: <class 'sodapy.socrata.Socrata'>


### Step 2: Setup your Google BigQuery variables (ACTION REQUIRED HERE)

In [6]:
key_path = r'/Users/naseemdabiran/Desktop/BARUCH/SPRING 2023/CIS 9440 - Data Warehousing and Analytics/Final Project/fundamental-run-375818-8e10844379a7.json'

In [7]:
credentials = service_account.Credentials.from_service_account_file(key_path,
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"],)
bigquery_client = bigquery.Client(credentials = credentials,
                                 project = credentials.project_id)

print(f"bigquery client name is: {bigquery_client}")
print(f"bigquery client data type is: {type(bigquery_client)}")

bigquery client name is: <google.cloud.bigquery.client.Client object at 0x7fa8df6cac70>
bigquery client data type is: <class 'google.cloud.bigquery.client.Client'>


In [8]:
dataset_id = 'fundamental-run-375818.collisions_project' 

dataset_id = dataset_id.replace(':', '.')
print(f"your dataset_id is: {dataset_id}")

your dataset_id is: fundamental-run-375818.collisions_project


### Step 3: Extract data

In [9]:
import time
start = time.time()

# Get the total number of records in each of our the datasets
for ds in [dataset_collisions]:
    total_record_count = nyc_open_data_client.get(ds, select = "COUNT(collision_id)")
    metadata = nyc_open_data_client.get_metadata(ds)
    print(f"total records in {ds}: {total_record_count[0]['COUNT_collision_id']}")
    print('='*10)
    print(f"columns in dataset: {[x['name'] for x in metadata['columns']]}")

end=time.time()
print(f"function completed in {round(end-start, 1)} seconds")

total records in h9gi-nx95: 2024478
columns in dataset: ['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']
function completed in 14.0 seconds


In [10]:
# filter for only collisions that occured since Jan 01, 2021
target_record_count = nyc_open_data_client.get(dataset_collisions,
                                               where = "crash_date >= '2021-01-01' AND crash_date < '2023-04-05'",
                                               select= "COUNT(*)")

print(f"target records in {dataset_collisions}: {int(target_record_count[0]['COUNT'])}")

target records in h9gi-nx95: 237950


In [11]:
def extract_socrata_data(chunk_size = 1000,
                         data_set = None,
                         where = None):
    
    # measure time this function takes
    import time
    start_time = time.time()
    
    # get total number or records
    if where == None:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     select= "COUNT(*)")[0]["COUNT"])
    else:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     where = where,
                                                     select= "COUNT(*)")[0]["COUNT"])
    
    start = 0                   
    results = []                

    while True:

        if where == None:
            results.extend(nyc_open_data_client.get(data_set,
                                                    offset = start,
                                                    limit = chunk_size))
            
        elif where != None:
            results.extend(nyc_open_data_client.get(data_set,
                                                    where = where,
                                                    offset = start,
                                                    limit = chunk_size))
        # update the starting record number
        start = start + chunk_size

        # exit loop when all records grabbed
        if (start > total_records):
            break

    # convert the list into a pandas data frame
    data = pd.DataFrame.from_records(results)

    end_time = time.time()
    print(f"function took {round(end_time - start_time, 1)} seconds")

    print(f"the shape of your dataframe is: {data.shape}")
    return data

In [12]:
data = extract_socrata_data(chunk_size = 1000,
                            data_set = dataset_collisions,
                            where = "crash_date >= '2021-01-01' AND crash_date < '2023-04-05'")

function took 189.1 seconds
the shape of your dataframe is: (237950, 29)


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237950 entries, 0 to 237949
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   crash_date                     237950 non-null  object
 1   crash_time                     237950 non-null  object
 2   latitude                       218524 non-null  object
 3   longitude                      218524 non-null  object
 4   location                       218524 non-null  object
 5   on_street_name                 173530 non-null  object
 6   number_of_persons_injured      237949 non-null  object
 7   number_of_persons_killed       237950 non-null  object
 8   number_of_pedestrians_injured  237950 non-null  object
 9   number_of_pedestrians_killed   237950 non-null  object
 10  number_of_cyclist_injured      237950 non-null  object
 11  number_of_cyclist_killed       237950 non-null  object
 12  number_of_motorist_injured     237950 non-nu

### Adding Weather Data

In [14]:
weather_df = pd.read_csv(r'weather_nyc.csv')

In [15]:
weather_df.head()

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2021-01-01,2.5,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-02,5.5,2.0,11.0,6.1,NaN,316.0,17.5,NaN,1012.5,NaN
2,2021-01-03,2.3,2.0,3.0,5.2,NaN,41.0,15.6,NaN,1017.3,NaN
3,2021-01-04,3.4,2.0,6.0,0.6,NaN,3.0,11.2,NaN,1015.0,NaN
4,2021-01-05,3.5,2.0,5.0,0.2,NaN,356.0,9.5,NaN,1013.6,NaN


In [16]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    825 non-null    object 
 1   tavg    822 non-null    float64
 2   tmin    822 non-null    float64
 3   tmax    822 non-null    float64
 4   prcp    821 non-null    float64
 5   snow    0 non-null      float64
 6   wdir    821 non-null    float64
 7   wspd    821 non-null    float64
 8   wpgt    0 non-null      float64
 9   pres    821 non-null    float64
 10  tsun    0 non-null      float64
dtypes: float64(10), object(1)
memory usage: 71.0+ KB


In [17]:
# update date columns to date data types
data["crash_date"] = pd.to_datetime(data["crash_date"])
weather_df["date"] = pd.to_datetime(weather_df["date"])

In [18]:
# merge the dataframes on date
data = data.merge(weather_df,
                         how = 'inner',
                         left_on = "crash_date",
                         right_on = "date",
                         suffixes=('', '_y'))
data.head()

,crash_date,crash_time,latitude,longitude,location,on_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,...,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2021-01-01,20:00,40.83398,-73.82635,"{'latitude': '40.83398', 'longitude': '-73.826...",BRUCKNER EXPRESSWAY,0,0,0,0,...,2.5,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01,5:28,40.6873,-73.973656,"{'latitude': '40.6873', 'longitude': '-73.9736...",LAFAYETTE AVENUE,0,0,0,0,...,2.5,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01,6:00,NaN,NaN,NaN,WEST SHORE EXPRESSWAY,0,0,0,0,...,2.5,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01,19:30,40.8827000,-73.8927300,"{'latitude': '40.8827', 'longitude': '-73.89273'}",SEDGWICK AVENUE,0,0,0,0,...,2.5,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01,7:40,40.6379100,-73.9786400,"{'latitude': '40.63791', 'longitude': '-73.978...",CORTELYOU ROAD,0,0,0,0,...,2.5,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Step 4: Data Profiling

In [19]:
data.columns

Index(['crash_date', 'crash_time', 'latitude', 'longitude', 'location',
       'on_street_name', 'number_of_persons_injured',
       'number_of_persons_killed', 'number_of_pedestrians_injured',
       'number_of_pedestrians_killed', 'number_of_cyclist_injured',
       'number_of_cyclist_killed', 'number_of_motorist_injured',
       'number_of_motorist_killed', 'contributing_factor_vehicle_1',
       'collision_id', 'vehicle_type_code1', 'contributing_factor_vehicle_2',
       'vehicle_type_code2', 'borough', 'zip_code', 'off_street_name',
       'cross_street_name', 'contributing_factor_vehicle_3',
       'vehicle_type_code_3', 'contributing_factor_vehicle_4',
       'contributing_factor_vehicle_5', 'vehicle_type_code_4',
       'vehicle_type_code_5', 'date', 'tavg', 'tmin', 'tmax', 'prcp', 'snow',
       'wdir', 'wspd', 'wpgt', 'pres', 'tsun'],
      dtype='object')

In [20]:
# subset for only needed columns
data = data[['crash_date', 
             'crash_time', 
             'borough', 
             'zip_code',
             'on_street_name',
             'number_of_persons_injured', 
             'number_of_persons_killed',
             'contributing_factor_vehicle_1', 
             'contributing_factor_vehicle_2',
             'vehicle_type_code1', 
             'vehicle_type_code2',
             'contributing_factor_vehicle_3',
             'contributing_factor_vehicle_4', 
             'contributing_factor_vehicle_5',
             'vehicle_type_code_3', 
             'vehicle_type_code_4', 
             'vehicle_type_code_5', 
             "tavg",
             "prcp",
             "wspd"]]

In [21]:
# create and run a function to ceate data profiling dataframe

def create_data_profiling_df(data):
    
    # create an empty dataframe to gather information about each column
    data_profiling_df = pd.DataFrame(columns = ["column_name",
                                                "column_type",
                                                "unique_values",
                                                "duplicate_values",
                                                "null_values",
                                                "non_null_values"])

    # loop through each column to add rows to the data_profiling_df dataframe
    for column in data.columns:

        info_dict = {}

        try:
            info_dict["column_name"] = column
            info_dict["column_type"] = data[column].dtypes
            info_dict["unique_values"] = len(data[column].unique())
            info_dict["duplicate_values"] = data[column].count() - len(data[column].dropna().unique())
            info_dict["null_values"] = data[column].isna().sum()
            info_dict["non_null_values"] = data[column].count()

        except:
            print(f"unable to read column: {column}, you may want to drop this column")

        data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)

    data_profiling_df.sort_values(by = ['unique_values', "non_null_values"],
                                  ascending = [False, False],
                                  inplace=True)
    
    return data_profiling_df

In [22]:
data_profiling_df = create_data_profiling_df(data)
data_profiling_df

<ipython-input-21-7c51118b259e>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
<ipython-input-21-7c51118b259e>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
<ipython-input-21-7c51118b259e>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
<ipython-input-21-7c51118b259e>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
<ipython-input-2

,column_name,column_type,unique_values,duplicate_values,null_values,non_null_values
4,on_street_name,object,8278,165253,64420,173530
1,crash_time,object,1440,236510,0,237950
0,crash_date,datetime64[ns],823,237127,0,237950
10,vehicle_type_code2,object,699,156796,80456,157494
9,vehicle_type_code1,object,587,234242,3122,234828
17,tavg,float64,325,236895,731,237219
3,zip_code,object,220,156403,81328,156622
19,wspd,float64,169,236794,988,236962
18,prcp,float64,153,236810,988,236962
14,vehicle_type_code_3,object,113,21733,216105,21845


In [23]:
data["borough"].unique()

array([nan, 'BRONX', 'BROOKLYN', 'QUEENS', 'MANHATTAN', 'STATEN ISLAND'],
      dtype=object)

In [24]:
data['contributing_factor_vehicle_1'].unique()

array(['Pavement Slippery', 'Unspecified', 'Fell Asleep', nan,
       'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
       'Other Vehicular', 'Passing Too Closely', 'Following Too Closely',
       'Turning Improperly', 'Unsafe Speed',
       'Driver Inattention/Distraction', 'Alcohol Involvement',
       'Accelerator Defective', 'Failure to Yield Right-of-Way',
       'Backing Unsafely', 'Passing or Lane Usage Improper',
       'View Obstructed/Limited', 'Tire Failure/Inadequate',
       'Driver Inexperience', 'Steering Failure',
       'Reaction to Uninvolved Vehicle', 'Fatigued/Drowsy',
       'Aggressive Driving/Road Rage', 'Traffic Control Disregarded',
       'Unsafe Lane Changing', 'Drugs (illegal)', 'Passenger Distraction',
       'Illnes', 'Oversized Vehicle', 'Glare', 'Lost Consciousness',
       'Outside Car Distraction', 'Obstruction/Debris', 'Animals Action',
       'Traffic Control Device Improper/Non-Working',
       'Pavement Defective', 'Brakes Defective',
  

In [25]:
data["vehicle_type_code1"].unique()

array(['Station Wagon/Sport Utility Vehicle', 'Sedan', nan, 'Taxi',
       'Convertible', 'Ambulance', 'Bike', 'Pick-up Truck',
       'Tractor Truck Diesel', 'Bus', 'E-Scooter', '4 dr sedan',
       'Box Truck', 'LIMO', 'PK', 'E-Bike', 'Flat Rack', 'FIRE TRUCK',
       'Garbage or Refuse', 'Motorcycle', 'Dump', 'FDNY TRUCK',
       'Chassis Cab', 'DUMP TRUCK', 'Van', 'Beverage Truck',
       'Armored Truck', 'AMBULANCE', 'Lunch Wagon', 'Carry All',
       'Flat Bed', 'Tow Truck / Wrecker', 'Tanker', 'PICK UP', 'G PSD',
       'Moped', 'Unk', 'Scooter', 'passenger', 'Refrigerated Van',
       'box truck', 'Motorscooter', 'Tractor Truck Gasoline', 'TRUCK',
       'ambulance', 'ROAD SWEEP', 'ford', 'Lift Boom', '3-Door', 'ambu',
       'UTIL', 'fire truck', 'Open Body', 'Glass Rack', 'EMS/VAN',
       'UNKNOWN', 'truck', 'FIRE ENGIN', 'Motorbike', 'SPECIAL CO',
       'Pick up', 'Tow Truck', 'UTILITY VE', 'truck trai', 'TRAILER',
       'Quality tr', 'van', 'Concrete Mixer', 'box', 'Hors

In [26]:
data["vehicle_type_code2"].unique()

array([nan, 'Sedan', 'Taxi', 'Station Wagon/Sport Utility Vehicle',
       'Pick-up Truck', 'Bus', 'UNKNOWN', 'Ambulance', 'Bike', 'Flat Bed',
       'E-Scooter', 'unk', 'Chassis Cab', 'Tanker', 'Box Truck',
       'Convertible', 'PK', 'E-Bike', 'FDNY',
       'Enclosed Body - Removable Enclosure', 'Dump', 'Motorcycle',
       'Tractor Truck Diesel', 'Van', 'Tractor Truck Gasoline', 'TRAILER',
       'Moped', 'Garbage or Refuse', 'E-BIKE', 'Armored Truck', 'EMS',
       'Carry All', '4 dr sedan', 'Tow Truck / Wrecker', 'van',
       'Snow Plow', 'RV', 'INTE TRUCK', 'Lift Boom', 'unknown', 'scooter',
       'Scooter', 'ambulance', 'FIRE TRUCK', 'Motorscooter',
       'Refrigerated Van', 'Stake or Rack', 'Bobcat', 'LIMO',
       'USPS TRUCK', 'FDNY Firet', 'Minibike', 'MTA', 'AMBULANCE',
       'FDNY LADDE', 'POWER SHOV', 'fire truck', 'GOLF KART', 'FDNY FIRE',
       'Open Body', 'UNK', 'Motorbike', 'DUMP', 'SPC', 'EXCAVATOR',
       'freight tr', 'Command Po', 'TRACTOR TR', 'BOX TRUCK'

In [27]:
data["vehicle_type_code_3"].unique()

array([nan, 'Sedan', 'Station Wagon/Sport Utility Vehicle',
       'Pick-up Truck', 'Taxi', 'Truck', 'E-Scooter', 'Dump', 'Box Truck',
       'Tractor Truck Diesel', 'Chassis Cab', 'Flat Bed', 'Bus',
       'Convertible', 'Motorcycle', 'Motorscooter', 'Van', 'E-Bike',
       'Tanker', 'Bike', 'POWER SHOV', 'semitraile', 'Refrigerated Van',
       'UNKNOWN', 'Garbage or Refuse', 'UNK', 'unknown', 'TRAILER',
       'Sierra', '3-Door', 'Ambulance', 'Moped', '4 dr sedan',
       'Carry All', 'VS2', 'PICKUP', 'LIMO', 'Fdny fire', 'motor home',
       'Motorbike', 'DIRT BIKE', 'Tow Truck / Wrecker', 'PK',
       'Tractor Truck Gasoline', 'Armored Truck', 'DUMP', 'FIRE TRUCK',
       'TOW TRUCK', 'Concrete Mixer', 'FDNY FIRE', 'Lift Boom',
       'School bus', 'NYC AMBULA', 'MOPED', 'AMBULANCE', 'sprinter', 'C1',
       'School Bus', 'BOX TRUCK', 'Multi-Wheeled Vehicle', 'Budget tru',
       'COMMERCIAL', 'Open Body', 'Moped scoo', 'STATION WA',
       'vehicle tr', 'US GOVERME', 'NYC SANITA'

In [28]:
data["vehicle_type_code_4"].unique()

array([nan, 'Sedan', 'Station Wagon/Sport Utility Vehicle',
       'Pick-up Truck', 'Taxi', '4 dr sedan', 'Motorscooter',
       'Garbage or Refuse', 'Van', 'Tractor Truck Diesel', 'Box Truck',
       'Bike', 'PK', 'Bus', 'SNOWMOBILE', 'Motorcycle', 'WORK VAN',
       'Dump', 'bus', 'E-Scooter', 'E-Bike', 'van', 'Convertible',
       'DELIVERY T', 'Moped', 'Pedicab', 'NYC TRANSI', 'Flat Bed', 'UNK',
       'School Bus', 'Firetruck', 'Beverage Truck',
       'Tractor Truck Gasoline', 'Tow Truck / Wrecker', 'Tanker', 'LIMO',
       'SEDAN', 'ECONOLINE', 'Carry All', 'SUBURBAN', 'Lift Boom',
       '3-Door', 'Courier', 'Trailor', 'AMBULANCE', 'Bulk Agriculture',
       'Multi-Wheeled Vehicle', 'BOAT', 'PICK UP TR', 'Ambulance',
       'Tractor Tr', 'Motorbike', 'SEMI-TRUCK', 'Chassis Cab'],
      dtype=object)

In [29]:
data["vehicle_type_code_5"].unique()

array([nan, 'Sedan', 'Station Wagon/Sport Utility Vehicle', 'Taxi',
       'trailer', 'Motorscooter', 'Van', 'Tractor Truck Diesel',
       'Pick-up Truck', 'Bus', 'PK', 'SNOWMOBILE', 'Box Truck',
       'E-Scooter', 'C3', 'Motorcycle', 'Moped', 'Bike', 'Dump',
       'Tow Truck / Wrecker', 'TRACTOR TR', 'Lunch Wagon', 'TRAILOR',
       'Tractor Truck Gasoline', 'Convertible', 'E-Bike', 'Truck',
       'Armored Truck', 'E-BIKE', 'TRAILER', 'UHAUL TRAI'], dtype=object)

### Step 5: Data Cleansing

Drop the rows where the on_street_name is null

Drop the rows where the vehicle_type_code1 is null.

Keep the rows where vehicle_type_code2, vehicle_type_code3, vehicle_type_code4, vehicle_type_code5 are null as there may not be up to 5 cars involved in a collision. 

Drop the rows where the contributing_factor_vehicle_1 is null. 

Keep the rows where contributing_factor_vehicle_2, contributing_factor_vehicle_3, contributing_factor_vehicle_4, contributing_factor_vehicle_5 are null as there may not be up to 5 cars involved in a collision and only 1 car may be at fault

Drop the rows where the zip_code and borough is null. 

Drop the columns where the borough is nan.

Drop the rows where the number_of_persons_injured is null. 

Replace the different vehicle_type_codes if they match with something similar, such as 'Fire truck' and 'FDNY TRUCK' will become 'Fire truck'

Drop duplicate rows, and update the data types to match out physical model.

In [30]:
# How the dataset currently looks
data.head(5)

,crash_date,crash_time,borough,zip_code,on_street_name,number_of_persons_injured,number_of_persons_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,tavg,prcp,wspd
0,2021-01-01,20:00,NaN,NaN,BRUCKNER EXPRESSWAY,0,0,Pavement Slippery,NaN,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN
1,2021-01-01,5:28,NaN,NaN,LAFAYETTE AVENUE,0,0,Unspecified,Unspecified,Sedan,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN
2,2021-01-01,6:00,NaN,NaN,WEST SHORE EXPRESSWAY,0,0,Fell Asleep,NaN,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN
3,2021-01-01,19:30,BRONX,10463,SEDGWICK AVENUE,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN
4,2021-01-01,7:40,BROOKLYN,11218,CORTELYOU ROAD,0,0,Unspecified,Unspecified,Station Wagon/Sport Utility Vehicle,Taxi,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN


In [31]:
# drop rows with null values in specific columns
data = data.dropna(subset = ['on_street_name',
                             'zip_code',
                             'borough', 
                             'number_of_persons_injured', 
                             'vehicle_type_code1', 
                             'contributing_factor_vehicle_1', 
                             'tavg', 
                             'wspd', 
                             'prcp'])

In [32]:
# Make the number_of_persons_injured and number_of_persons_killed columns into integers
for column in ['number_of_persons_injured', 'number_of_persons_killed', 'zip_code']:
        data[column] = data[column].astype(int)


data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 92412 entries, 257 to 237945
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_date                     92412 non-null  datetime64[ns]
 1   crash_time                     92412 non-null  object        
 2   borough                        92412 non-null  object        
 3   zip_code                       92412 non-null  int64         
 4   on_street_name                 92412 non-null  object        
 5   number_of_persons_injured      92412 non-null  int64         
 6   number_of_persons_killed       92412 non-null  int64         
 7   contributing_factor_vehicle_1  92412 non-null  object        
 8   contributing_factor_vehicle_2  73000 non-null  object        
 9   vehicle_type_code1             92412 non-null  object        
 10  vehicle_type_code2             63875 non-null  object        
 11  contributing

In [33]:
# rename columns
data.rename({"tavg": "avg_temp",
             "prcp": "precipitation",
             "wspd": "wind_speed"},
             axis = 'columns',
             inplace = True)

In [34]:
# convert temperature column to farenheit
data["avg_temp"] = (data["avg_temp"]*1.8) +32

# convert precipitation to inches
data["precipitation"] = data["precipitation"]/25.4

# convert wind speed to mph
data["wind_speed"] = data["wind_speed"]/ 1.609


In [35]:
data.describe()

,zip_code,number_of_persons_injured,number_of_persons_killed,avg_temp,precipitation,wind_speed
count,92412.000000,92412.000000,92412.000000,92412.000000,92412.000000,92412.000000
mean,10905.996169,0.564267,0.003030,56.298635,0.129484,6.366245
std,522.741666,0.825276,0.056896,16.466459,0.313012,2.344246
min,10000.000000,0.000000,0.000000,11.300000,0.000000,1.615911
25%,10456.000000,0.000000,0.000000,42.620000,0.000000,4.847732
50%,11208.000000,0.000000,0.000000,56.840000,0.000000,5.966439
75%,11355.000000,1.000000,0.000000,70.520000,0.102362,7.520199
max,11697.000000,23.000000,3.000000,86.540000,3.173228,21.628341


In [36]:
# add temperature_category
data["temperature_category"] = pd.cut(data["avg_temp"], [-np.inf, 40, 60, np.inf],
                              labels=['cold', 'moderate', 'hot'])

# add raining_flag
data['raining_flag'] = data['precipitation'] >0

# add windy_flag
data['windy_flag'] = data['wind_speed'] >= 7.5

In [37]:
# find number of duplicate rows
print(f"number of duplicate rows: {len(data[data.duplicated()])}")

number of duplicate rows: 291


In [38]:
# drop duplicate rows based on entire row
data = data.drop_duplicates(keep = 'first')

print(f"number of rows after duplicates dropped: {len(data)}")

number of rows after duplicates dropped: 92121


In [39]:
# Fixing the vehicle codes that have the same meaning 

#First concat all of the vehicle codes into one array and find the unique values
vehicle_codes = data[['vehicle_type_code2', 'vehicle_type_code1', 'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5']]
merged_df = pd.DataFrame(columns=['merged_column'])

# Append the values from each existing column to the merged column
for col in vehicle_codes.columns:
    temp_df = pd.DataFrame(vehicle_codes[col].values, columns=['merged_column'])
    merged_df = merged_df.append(temp_df, ignore_index=True)

#Drop duplicates
merged_df.dropna(subset = ['merged_column'], inplace=True) 


<ipython-input-39-ce9b86efb483>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(temp_df, ignore_index=True)
<ipython-input-39-ce9b86efb483>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(temp_df, ignore_index=True)
<ipython-input-39-ce9b86efb483>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(temp_df, ignore_index=True)
<ipython-input-39-ce9b86efb483>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(temp_df, ignore_index=True)
<ipython-input-39-ce9b86efb483>:10: FutureWarning: The frame.append method is deprecated

In [40]:
# Number of unique values
print(len(merged_df['merged_column'].unique()))
# All the unique values and their counts
print(merged_df['merged_column'].unique())

533
['E-Scooter' 'Bike' 'Sedan' 'Station Wagon/Sport Utility Vehicle'
 'Pick-up Truck' 'PK' 'E-Bike' 'FDNY' 'Bus' 'Flat Bed' 'Dump' 'Taxi'
 'Tractor Truck Gasoline' 'TRAILER' 'Tractor Truck Diesel' 'Box Truck'
 'Moped' 'Garbage or Refuse' 'Van' 'E-BIKE' 'Motorcycle' '4 dr sedan'
 'Tow Truck / Wrecker' 'Ambulance' 'Snow Plow' 'Carry All' 'Lift Boom'
 'Convertible' 'unknown' 'scooter' 'FIRE TRUCK' 'Motorscooter' 'Tanker'
 'AMBULANCE' 'fire truck' 'FDNY FIRE' 'LIMO' 'Motorbike' 'DUMP'
 'Chassis Cab' 'TRACTOR TR' 'BOX TRUCK' 'Concrete Mixer' 'road sweep'
 'UNKNOWN' 'Unknown' '3-Door' 'usps self' 'Flat Rack' 'NYC SANITA' 'subn'
 'sanitation' 'Open Body' 'School Bus' 'TRACTOR' 'FDNY TRUCK' 'Plow  truc'
 'EXCAVATOR' 'Armored Truck' 'PLOW' 'FORKLIFT' 'trailer' 'GARBAGE TR'
 'GAS SCOOTE' 'DELV' 'TRUCK' 'PSD' 'Beverage Truck' 'mta bus  4'
 'AMbulance' 'PUMPER' '2 dr sedan' 'SANITATION' 'Trailer' 'MOPED'
 'Minicycle' 'SCHOOL VAN' 'USPS MAIL' 'moped' 'van' 'UNK' 'box truck'
 'REVEL SCOO' 'truck' '

In [41]:
# Drop rows where vehicle type is unknown or N/A
for col in ['vehicle_type_code1','vehicle_type_code2', 'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5']:
    data[col] = data[col].str.upper()
    data = data[data[col] != 'UNKNOWN']
    data = data[data[col] != 'UNKNOW']
    data = data[data[col] != 'UNK']
    data = data[data[col] != 'LEFT SCENE']
    data = data[data['vehicle_type_code1'] != 'N/A']


In [42]:
# Standardize vehicle codes

import re

for col in ['vehicle_type_code2', 'vehicle_type_code1', 'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5']:
    
    data[col] = data[col].str.replace(r'EM|AMB', 'AMBULANCE', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'FD|FIRE', 'FIRE TRUCK', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'GAR|DUMP|SANI', 'GARBAGE TRUCK', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'SED|E350|4dr s|3-Door', 'SEDAN', flags=re.IGNORECASE)  
    data[col] = data[col].str.replace(r'WHEELCHA', 'WHEELCHAIR', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'CHASSIS', 'CHASSIS CAB', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'REVEL|TAX|LIMO|TL|CAB', 'TAXI AND LIMOUSINE', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'SCHO|YELLOW', 'SCHOOL BUS', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'PAS|PC', 'PASSENGER VEHICLE', flags=re.IGNORECASE)
    data[col] = data[col].replace('OPEN BODY', 'CONVERTIBLE')
    data[col] = data[col].replace(['PEPSI DELI' , 'LUNCH WAGON'], 'FOOD CART')
    data[col] = data[col].str.replace(r'BU|MT|TRANSI', 'BUS', flags=re.IGNORECASE)
    data[col] = data[col].replace(['NYPD VAN','POLICE REP'], 'NYPD')
    data[col] = data[col].str.replace(r'STATION WA|MINIVAN|MINI VAN', 'SUV', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'GOV|CITY|DOT', 'GOVERNMENT VEHICLE', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'CAMPER|HOUSE|HOME|RV', 'MOBILE HOME', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'PICK|150|PK|RAM|F550', 'PICK-UP TRUCK', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'VA|ECO|SPRINT|EC2|PRO MASTER|COU|CARG|ENC|DELIVERY V', 'VAN', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'MAIL|POST|USP', 'POSTAL VEHICLE', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'TRAIL', 'TRAILER', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'TOW', 'TOW TRUCK', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'CAT|POWER SHOV|FORK|CONST|CONT|CRANE|MIX|CEMENT|LADD|LIFT|EXCAVAT|BULL|BULK|BACK|CARRY ALL|PUMP|DRILL|GATOR|JOHNDE', 'INDUSTRIAL VEHICLE', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'UTIL|MAINTEN|CON ED', 'UTILITY VEHICLE', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'SWE|CLE|PLOW', 'SWEEPER OR SNOW PLOW', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'BIK|BIC|UNICY|MINICY|SKATE|BOARD', 'BICYCLE,  UNICYCLE, or SKATEBOARD', flags=re.IGNORECASE)
    data[col] = data[col].replace('FLAT', 'FLATBED')
    data[col] = data[col].str.replace(r'MOTOR B|GAS MO|GAS SCO|GAS BI|YAMAHA|MOTORB|MOTORC', 'MOTORCYCLE', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'SC|MOP|MOBILITY|STAND|ELECTRIC', 'SCOOTER OR MOPED', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'TRAC|MACK|SEMI|TR|BOX|TAN|DET|PAL|FR|STRA|U|FED|REF|DL|DEL|TK', 'TRUCK', flags=re.IGNORECASE)
    data[col] = data[col].str.replace(r'COM', 'COMMERCIAL VEHICLE', flags=re.IGNORECASE)

<ipython-input-42-bc4fa9bac733>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r'EM|AMB', 'AMBULANCE', flags=re.IGNORECASE)
<ipython-input-42-bc4fa9bac733>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r'FD|FIRE', 'FIRE TRUCK', flags=re.IGNORECASE)
<ipython-input-42-bc4fa9bac733>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r'GAR|DUMP|SANI', 'GARBAGE TRUCK', flags=re.IGNORECASE)
<ipython-input-42-bc4fa9bac733>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(r'SED|E350|4dr s|3-Door', 'SEDAN', flags=re.IGNORECASE)
<ipython-input-42-bc4fa9bac733>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  dat

In [43]:
#Check how many different unique vehicle codes now are 
vehicle_codes = data[['vehicle_type_code2', 'vehicle_type_code1', 'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5']]
merged_df = pd.DataFrame(columns=['merged_column'])

for col in vehicle_codes.columns:
    temp_df = pd.DataFrame(vehicle_codes[col].values, columns=['merged_column'])
    merged_df = merged_df.append(temp_df, ignore_index=True)

#Drop duplicates
merged_df.dropna(subset = ['merged_column'], inplace=True) 
print(len(merged_df['merged_column'].unique()))

358


<ipython-input-43-a227312d30d9>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(temp_df, ignore_index=True)
<ipython-input-43-a227312d30d9>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(temp_df, ignore_index=True)
<ipython-input-43-a227312d30d9>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(temp_df, ignore_index=True)
<ipython-input-43-a227312d30d9>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(temp_df, ignore_index=True)
<ipython-input-43-a227312d30d9>:7: FutureWarning: The frame.append method is deprecated and 

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92059 entries, 257 to 237945
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_date                     92059 non-null  datetime64[ns]
 1   crash_time                     92059 non-null  object        
 2   borough                        92059 non-null  object        
 3   zip_code                       92059 non-null  int64         
 4   on_street_name                 92059 non-null  object        
 5   number_of_persons_injured      92059 non-null  int64         
 6   number_of_persons_killed       92059 non-null  int64         
 7   contributing_factor_vehicle_1  92059 non-null  object        
 8   contributing_factor_vehicle_2  72738 non-null  object        
 9   vehicle_type_code1             92059 non-null  object        
 10  vehicle_type_code2             63618 non-null  object        
 11  contributing

### Step 6:  Create Dimensions and Fact Tables

### Create Location Dimension



In [45]:
# first, copy the entire table
location_dim = data.copy()

In [46]:
# look at all the columns in location_dim
location_dim.columns

Index(['crash_date', 'crash_time', 'borough', 'zip_code', 'on_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'vehicle_type_code1', 'vehicle_type_code2',
       'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4',
       'contributing_factor_vehicle_5', 'vehicle_type_code_3',
       'vehicle_type_code_4', 'vehicle_type_code_5', 'avg_temp',
       'precipitation', 'wind_speed', 'temperature_category', 'raining_flag',
       'windy_flag'],
      dtype='object')

In [47]:
# second, subset for only the wanted columns in the dimension 
location_dim = location_dim[["borough", 'zip_code', 'on_street_name']]

In [48]:
# third, drop duplicate rows in dimension
unique_row = ["borough", 'zip_code', 'on_street_name']
location_dim = location_dim.drop_duplicates(subset = unique_row, keep = 'first')
location_dim = location_dim.reset_index(drop = True)
location_dim

,borough,zip_code,on_street_name
0,QUEENS,11377,SKILLMAN AVENUE
1,BROOKLYN,11249,SOUTH 10 STREET
2,BRONX,10468,Father Zeiser place
3,BROOKLYN,11226,LENOX ROAD
4,BRONX,10452,WEST 162 STREET
...,...,...,...
12633,STATEN ISLAND,10301,ONEIDA AVENUE
12634,BRONX,10465,KEARNEY AVENUE
12635,BRONX,10465,HOLLYWOOD AVENUE
12636,QUEENS,11208,LINDEN BOULEVARD


In [49]:
# fourth, add a surrogate key
location_dim.insert(0, "location_id", range(50, 50 + len(location_dim)))
location_dim

,location_id,borough,zip_code,on_street_name
0,50,QUEENS,11377,SKILLMAN AVENUE
1,51,BROOKLYN,11249,SOUTH 10 STREET
2,52,BRONX,10468,Father Zeiser place
3,53,BROOKLYN,11226,LENOX ROAD
4,54,BRONX,10452,WEST 162 STREET
...,...,...,...,...
12633,12683,STATEN ISLAND,10301,ONEIDA AVENUE
12634,12684,BRONX,10465,KEARNEY AVENUE
12635,12685,BRONX,10465,HOLLYWOOD AVENUE
12636,12686,QUEENS,11208,LINDEN BOULEVARD


In [50]:
# fifth, add the surrogate key to the data table
data = data.merge(location_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')

data.head(100)

,crash_date,crash_time,borough,zip_code,on_street_name,number_of_persons_injured,number_of_persons_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,...,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,avg_temp,precipitation,wind_speed,temperature_category,raining_flag,windy_flag,location_id
0,2021-01-02,10:45,QUEENS,11377,SKILLMAN AVENUE,1,0,Driver Inattention/Distraction,Driver Inattention/Distraction,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,...,NaN,NaN,NaN,41.90,0.240157,10.876321,moderate,True,True,50
1,2021-01-02,22:00,BROOKLYN,11249,SOUTH 10 STREET,0,0,Driver Inattention/Distraction,Unspecified,SEDANAN,...,NaN,NaN,NaN,41.90,0.240157,10.876321,moderate,True,True,51
2,2021-01-02,15:22,BRONX,10468,Father Zeiser place,0,0,Driver Inattention/Distraction,Unspecified,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,...,NaN,NaN,NaN,41.90,0.240157,10.876321,moderate,True,True,52
3,2021-01-02,17:45,BROOKLYN,11226,LENOX ROAD,1,0,Unspecified,Unspecified,SEDANAN,...,NaN,NaN,NaN,41.90,0.240157,10.876321,moderate,True,True,53
4,2021-01-02,14:55,BRONX,10452,WEST 162 STREET,1,0,Traffic Control Disregarded,Unspecified,SEDANAN,...,NaN,NaN,NaN,41.90,0.240157,10.876321,moderate,True,True,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021-01-03,19:00,QUEENS,11377,66 STREET,0,0,Driver Inattention/Distraction,Unspecified,SEDANAN,...,NaN,NaN,NaN,36.14,0.204724,9.695463,cold,True,True,145
96,2021-01-03,12:10,BRONX,10452,TOWNSEND AVENUE,0,0,Traffic Control Disregarded,Unspecified,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,...,NaN,NaN,NaN,36.14,0.204724,9.695463,cold,True,True,146
97,2021-01-03,17:21,BRONX,10457,PARK AVENUE,1,0,Other Vehicular,Other Vehicular,SEDANAN,...,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,36.14,0.204724,9.695463,cold,True,True,147
98,2021-01-03,0:00,BROOKLYN,11207,ATLANTIC AVENUE,0,0,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,NaN,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,...,NaN,NaN,NaN,36.14,0.204724,9.695463,cold,True,True,148


### Create Weather Dimension

In [51]:
# first, copy the entire table
weather_dim = data.copy()

In [52]:
# look at all the columns in complaint_dim
weather_dim.columns

Index(['crash_date', 'crash_time', 'borough', 'zip_code', 'on_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'vehicle_type_code1', 'vehicle_type_code2',
       'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4',
       'contributing_factor_vehicle_5', 'vehicle_type_code_3',
       'vehicle_type_code_4', 'vehicle_type_code_5', 'avg_temp',
       'precipitation', 'wind_speed', 'temperature_category', 'raining_flag',
       'windy_flag', 'location_id'],
      dtype='object')

In [53]:
# second, subset for only the wanted columns in the dimension
weather_dim = weather_dim[['temperature_category', 'raining_flag', 'windy_flag']]

In [54]:
# third, drop duplicate rows in dimension
unique_row = ['temperature_category', 'raining_flag', 'windy_flag'] 
weather_dim = weather_dim.drop_duplicates(subset = unique_row, keep = 'first')
weather_dim = weather_dim.reset_index(drop = True)
weather_dim

,temperature_category,raining_flag,windy_flag
0,moderate,True,True
1,cold,True,True
2,cold,True,False
3,cold,False,True
4,cold,False,False
5,moderate,False,False
6,moderate,False,True
7,moderate,True,False
8,hot,True,True
9,hot,False,False


In [55]:
# fourth, add a surrogate key
weather_dim.insert(0, "weather_id", range(100000, 100000 + len(weather_dim)))
weather_dim

,weather_id,temperature_category,raining_flag,windy_flag
0,100000,moderate,True,True
1,100001,cold,True,True
2,100002,cold,True,False
3,100003,cold,False,True
4,100004,cold,False,False
5,100005,moderate,False,False
6,100006,moderate,False,True
7,100007,moderate,True,False
8,100008,hot,True,True
9,100009,hot,False,False


In [56]:
# fifth, add the surrogate key to the data table
data = data.merge(weather_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')

data.head(100)

,crash_date,crash_time,borough,zip_code,on_street_name,number_of_persons_injured,number_of_persons_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,...,vehicle_type_code_4,vehicle_type_code_5,avg_temp,precipitation,wind_speed,temperature_category,raining_flag,windy_flag,location_id,weather_id
0,2021-01-02,10:45,QUEENS,11377,SKILLMAN AVENUE,1,0,Driver Inattention/Distraction,Driver Inattention/Distraction,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,...,NaN,NaN,41.90,0.240157,10.876321,moderate,True,True,50,100000
1,2021-01-02,22:00,BROOKLYN,11249,SOUTH 10 STREET,0,0,Driver Inattention/Distraction,Unspecified,SEDANAN,...,NaN,NaN,41.90,0.240157,10.876321,moderate,True,True,51,100000
2,2021-01-02,15:22,BRONX,10468,Father Zeiser place,0,0,Driver Inattention/Distraction,Unspecified,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,...,NaN,NaN,41.90,0.240157,10.876321,moderate,True,True,52,100000
3,2021-01-02,17:45,BROOKLYN,11226,LENOX ROAD,1,0,Unspecified,Unspecified,SEDANAN,...,NaN,NaN,41.90,0.240157,10.876321,moderate,True,True,53,100000
4,2021-01-02,14:55,BRONX,10452,WEST 162 STREET,1,0,Traffic Control Disregarded,Unspecified,SEDANAN,...,NaN,NaN,41.90,0.240157,10.876321,moderate,True,True,54,100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021-01-03,19:00,QUEENS,11377,66 STREET,0,0,Driver Inattention/Distraction,Unspecified,SEDANAN,...,NaN,NaN,36.14,0.204724,9.695463,cold,True,True,145,100001
96,2021-01-03,12:10,BRONX,10452,TOWNSEND AVENUE,0,0,Traffic Control Disregarded,Unspecified,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,...,NaN,NaN,36.14,0.204724,9.695463,cold,True,True,146,100001
97,2021-01-03,17:21,BRONX,10457,PARK AVENUE,1,0,Other Vehicular,Other Vehicular,SEDANAN,...,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,36.14,0.204724,9.695463,cold,True,True,147,100001
98,2021-01-03,0:00,BROOKLYN,11207,ATLANTIC AVENUE,0,0,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,NaN,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,...,NaN,NaN,36.14,0.204724,9.695463,cold,True,True,148,100001


### Create  Date Dimension

In [57]:
sql_query = """
            SELECT
              CONCAT (FORMAT_DATE("%Y",d),FORMAT_DATE("%m",d),FORMAT_DATE("%d",d)) as date_id,
              d AS full_date,
              FORMAT_DATE('%w', d) AS week_day,
              FORMAT_DATE('%A', d) AS day_name,
              EXTRACT(MONTH FROM d) AS month,
              FORMAT_DATE('%B', d) as month_name,
              EXTRACT(YEAR FROM d) AS year,

              (CASE WHEN FORMAT_DATE('%A', d) IN ('Sunday', 'Saturday') THEN 0 ELSE 1 END) AS day_is_weekday,
            FROM (
              SELECT
                *
              FROM
                UNNEST(GENERATE_DATE_ARRAY('2021-01-01', '2023-04-05', INTERVAL 1 DAY)) AS d )
            """

# store extracted data in new dataframe
date_dim = bigquery_client.query(sql_query).to_dataframe()

date_dim["date_id"] = date_dim["date_id"].astype(int)
date_dim["week_day"] = date_dim["week_day"].astype(int)

if len(date_dim) > 0:
    print(f"date dimension created successfully, shape of dimension: {date_dim.shape}")
else:
    print("date dimension FAILED")

date dimension created successfully, shape of dimension: (825, 8)


In [58]:
# create date_id column in the Fact Table
data['date_id'] = data['crash_date'].apply(lambda x: pd.to_datetime(x).strftime("%Y%m%d"))
data["date_id"] = data["date_id"].astype(int)

### Create Time Dimension

In [59]:
# create the time dimension
def create_time_dim():
    
    minutes_in_day = 60*24
    minutes = [pd.Timestamp('2023-04-05 00:00:00') + pd.Timedelta(minutes =i) for i in range(minutes_in_day)]
    
    df = pd.DataFrame({'time': minutes})
    df['fulltime'] = df['time'].dt.strftime("%H:%M")
    df['time_id'] = df['time'].dt.strftime("%H%M").astype(int)
    df['hour'] = df['fulltime'].str[:2].astype(int)
    df['min'] = df['fulltime'].str[3:].astype(int)
    
    # add time_of_day category
    df["time_of_day"] = pd.cut(df["time_id"], [-np.inf, 500, 1200, 1700, 2000, np.inf],
                              labels=['Early Morning', 'Morning', 'Afternoon', 'Evening', 'Night'])



    return df
    

In [60]:
# add time_id column to the data dataframe
time_dim = create_time_dim()
time_dim

,time,fulltime,time_id,hour,min,time_of_day
0,2023-04-05 00:00:00,00:00,0,0,0,Early Morning
1,2023-04-05 00:01:00,00:01,1,0,1,Early Morning
2,2023-04-05 00:02:00,00:02,2,0,2,Early Morning
3,2023-04-05 00:03:00,00:03,3,0,3,Early Morning
4,2023-04-05 00:04:00,00:04,4,0,4,Early Morning
...,...,...,...,...,...,...
1435,2023-04-05 23:55:00,23:55,2355,23,55,Night
1436,2023-04-05 23:56:00,23:56,2356,23,56,Night
1437,2023-04-05 23:57:00,23:57,2357,23,57,Night
1438,2023-04-05 23:58:00,23:58,2358,23,58,Night


In [61]:
# create time_id column in the Fact Table
data['time_id'] = data['crash_time'].apply(lambda x: pd.to_datetime(x).strftime("%H%M"))
data['time_id'] = data['time_id'].astype(int)

### Create Collisions Fact table

In [62]:
data

,crash_date,crash_time,borough,zip_code,on_street_name,number_of_persons_injured,number_of_persons_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,...,avg_temp,precipitation,wind_speed,temperature_category,raining_flag,windy_flag,location_id,weather_id,date_id,time_id
0,2021-01-02,10:45,QUEENS,11377,SKILLMAN AVENUE,1,0,Driver Inattention/Distraction,Driver Inattention/Distraction,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,...,41.90,0.240157,10.876321,moderate,True,True,50,100000,20210102,1045
1,2021-01-02,22:00,BROOKLYN,11249,SOUTH 10 STREET,0,0,Driver Inattention/Distraction,Unspecified,SEDANAN,...,41.90,0.240157,10.876321,moderate,True,True,51,100000,20210102,2200
2,2021-01-02,15:22,BRONX,10468,Father Zeiser place,0,0,Driver Inattention/Distraction,Unspecified,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,...,41.90,0.240157,10.876321,moderate,True,True,52,100000,20210102,1522
3,2021-01-02,17:45,BROOKLYN,11226,LENOX ROAD,1,0,Unspecified,Unspecified,SEDANAN,...,41.90,0.240157,10.876321,moderate,True,True,53,100000,20210102,1745
4,2021-01-02,14:55,BRONX,10452,WEST 162 STREET,1,0,Traffic Control Disregarded,Unspecified,SEDANAN,...,41.90,0.240157,10.876321,moderate,True,True,54,100000,20210102,1455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92054,2023-04-02,14:21,BRONX,10457,CLAREMONT PARKWAY,1,0,Traffic Control Disregarded,Unspecified,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,...,46.04,0.000000,9.819764,moderate,False,True,4897,100006,20230402,1421
92055,2023-04-02,0:40,QUEENS,11370,19 AVENUE,0,0,Driver Inattention/Distraction,Unspecified,PICK-TRUCKP TRUCKTRUCKCK-TRUCKP TRUCKTRUCKCK,...,46.04,0.000000,9.819764,moderate,False,True,4975,100006,20230402,40
92056,2023-04-02,13:00,MANHATTAN,10025,BROADWAY,0,0,Other Vehicular,Traffic Control Disregarded,SEDANAN,...,46.04,0.000000,9.819764,moderate,False,True,5684,100006,20230402,1300
92057,2023-04-02,5:08,MANHATTAN,10065,EAST 61 STREET,0,0,Unspecified,Unspecified,SEDANAN,...,46.04,0.000000,9.819764,moderate,False,True,7507,100006,20230402,508


In [63]:
# look at all the columns in data dataframe
data.columns

Index(['crash_date', 'crash_time', 'borough', 'zip_code', 'on_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'vehicle_type_code1', 'vehicle_type_code2',
       'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4',
       'contributing_factor_vehicle_5', 'vehicle_type_code_3',
       'vehicle_type_code_4', 'vehicle_type_code_5', 'avg_temp',
       'precipitation', 'wind_speed', 'temperature_category', 'raining_flag',
       'windy_flag', 'location_id', 'weather_id', 'date_id', 'time_id'],
      dtype='object')

In [64]:
# take a subset of fact_table for only the needed columns: which are keys and measures
collisions_fact_table = data[['location_id', 
                               'weather_id', 
                               'date_id', 
                               'time_id',  
                               'number_of_persons_injured', 
                               'number_of_persons_killed']]

collisions_fact_table

,location_id,weather_id,date_id,time_id,number_of_persons_injured,number_of_persons_killed
0,50,100000,20210102,1045,1,0
1,51,100000,20210102,2200,0,0
2,52,100000,20210102,1522,0,0
3,53,100000,20210102,1745,1,0
4,54,100000,20210102,1455,1,0
...,...,...,...,...,...,...
92054,4897,100006,20230402,1421,1,0
92055,4975,100006,20230402,40,0,0
92056,5684,100006,20230402,1300,0,0
92057,7507,100006,20230402,508,0,0


### Creating Contributing Factor Dimension table, Vehicles Dimension Table, and Vehicles Involved Fact Table


In [65]:
#make a new copy of the data that we will use just for the vehicles and 
v_and_cf_data =data.copy()

In [66]:
# look at all the columns in dataframe
v_and_cf_data.columns

Index(['crash_date', 'crash_time', 'borough', 'zip_code', 'on_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'vehicle_type_code1', 'vehicle_type_code2',
       'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4',
       'contributing_factor_vehicle_5', 'vehicle_type_code_3',
       'vehicle_type_code_4', 'vehicle_type_code_5', 'avg_temp',
       'precipitation', 'wind_speed', 'temperature_category', 'raining_flag',
       'windy_flag', 'location_id', 'weather_id', 'date_id', 'time_id'],
      dtype='object')

In [67]:
# take a subset of columns needed for vehicles involved fact table, vehicles dimension table, and contributing factors dimesnion table
v_and_cf_data = v_and_cf_data[['location_id', 
                               'date_id',
                               'time_id', 
                               'contributing_factor_vehicle_1', 
                               'contributing_factor_vehicle_2',
                               'contributing_factor_vehicle_3', 
                               'contributing_factor_vehicle_4',
                               'contributing_factor_vehicle_5', 
                               'vehicle_type_code1',
                               'vehicle_type_code2',
                               'vehicle_type_code_3',
                               'vehicle_type_code_4',
                               'vehicle_type_code_5']]
v_and_cf_data

,location_id,date_id,time_id,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,50,20210102,1045,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,E-SCOOTER OR MOPEDOOTER,NaN,NaN,NaN
1,51,20210102,2200,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,SEDANAN,NaN,NaN,NaN,NaN
2,52,20210102,1522,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,NaN,NaN,NaN,NaN
3,53,20210102,1745,Unspecified,Unspecified,NaN,NaN,NaN,SEDANAN,"BICYCLE, TRUCKNICYCLE, or SKATEBOARDE",NaN,NaN,NaN
4,54,20210102,1455,Traffic Control Disregarded,Unspecified,NaN,NaN,NaN,SEDANAN,SEDANAN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92054,4897,20230402,1421,Traffic Control Disregarded,Unspecified,NaN,NaN,NaN,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,NaN,NaN,NaN,NaN
92055,4975,20230402,40,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,PICK-TRUCKP TRUCKTRUCKCK-TRUCKP TRUCKTRUCKCK,NaN,NaN,NaN,NaN
92056,5684,20230402,1300,Other Vehicular,Traffic Control Disregarded,NaN,NaN,NaN,SEDANAN,SCOOTER OR MOPEDED,NaN,NaN,NaN
92057,7507,20230402,508,Unspecified,Unspecified,NaN,NaN,NaN,SEDANAN,TOW TRUCKTRUCKCK TRUCKTRUCKCK,NaN,NaN,NaN


In [68]:
#Concatenate the contributing factor and vehicle type, melt, then  then explode

#make new columns
v_and_cf_data['contributing_factor_vehicle_pair1'] = v_and_cf_data['contributing_factor_vehicle_1'].fillna('Unspecified') + '_' + v_and_cf_data['vehicle_type_code1']
v_and_cf_data['contributing_factor_vehicle_pair2'] = v_and_cf_data['contributing_factor_vehicle_2'].fillna('Unspecified') + '_' + v_and_cf_data['vehicle_type_code2']
v_and_cf_data['contributing_factor_vehicle_pair3'] = v_and_cf_data['contributing_factor_vehicle_3'].fillna('Unspecified') + '_' + v_and_cf_data['vehicle_type_code_3']
v_and_cf_data['contributing_factor_vehicle_pair4'] = v_and_cf_data['contributing_factor_vehicle_4'].fillna('Unspecified') + '_' + v_and_cf_data['vehicle_type_code_4']
v_and_cf_data['contributing_factor_vehicle_pair5'] = v_and_cf_data['contributing_factor_vehicle_5'].fillna('Unspecified') + '_' + v_and_cf_data['vehicle_type_code_5']

# drop the original columns
v_and_cf_data.drop(['contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
               'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4',
               'contributing_factor_vehicle_5', 'vehicle_type_code1', 'vehicle_type_code2',
               'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5'], axis=1, inplace=True)

# drop duplicates
v_and_cf_data = v_and_cf_data.drop_duplicates(keep = 'first')


v_and_cf_data


,location_id,date_id,time_id,contributing_factor_vehicle_pair1,contributing_factor_vehicle_pair2,contributing_factor_vehicle_pair3,contributing_factor_vehicle_pair4,contributing_factor_vehicle_pair5
0,50,20210102,1045,Driver Inattention/Distraction_STRUCKVGON/SPOR...,Driver Inattention/Distraction_E-SCOOTER OR MO...,NaN,NaN,NaN
1,51,20210102,2200,Driver Inattention/Distraction_SEDANAN,NaN,NaN,NaN,NaN
2,52,20210102,1522,Driver Inattention/Distraction_STRUCKVGON/SPOR...,NaN,NaN,NaN,NaN
3,53,20210102,1745,Unspecified_SEDANAN,"Unspecified_BICYCLE, TRUCKNICYCLE, or SKATEBO...",NaN,NaN,NaN
4,54,20210102,1455,Traffic Control Disregarded_SEDANAN,Unspecified_SEDANAN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
92054,4897,20230402,1421,Traffic Control Disregarded_STRUCKVGON/SPORT T...,NaN,NaN,NaN,NaN
92055,4975,20230402,40,Driver Inattention/Distraction_PICK-TRUCKP TRU...,NaN,NaN,NaN,NaN
92056,5684,20230402,1300,Other Vehicular_SEDANAN,Traffic Control Disregarded_SCOOTER OR MOPEDED,NaN,NaN,NaN
92057,7507,20230402,508,Unspecified_SEDANAN,Unspecified_TOW TRUCKTRUCKCK TRUCKTRUCKCK,NaN,NaN,NaN


In [69]:
# Melt dataframe
v_and_cf_data = pd.melt(v_and_cf_data, id_vars=['location_id', 'date_id', 'time_id'])

v_and_cf_data

,location_id,date_id,time_id,variable,value
0,50,20210102,1045,contributing_factor_vehicle_pair1,Driver Inattention/Distraction_STRUCKVGON/SPOR...
1,51,20210102,2200,contributing_factor_vehicle_pair1,Driver Inattention/Distraction_SEDANAN
2,52,20210102,1522,contributing_factor_vehicle_pair1,Driver Inattention/Distraction_STRUCKVGON/SPOR...
3,53,20210102,1745,contributing_factor_vehicle_pair1,Unspecified_SEDANAN
4,54,20210102,1455,contributing_factor_vehicle_pair1,Traffic Control Disregarded_SEDANAN
...,...,...,...,...,...
460275,4897,20230402,1421,contributing_factor_vehicle_pair5,NaN
460276,4975,20230402,40,contributing_factor_vehicle_pair5,NaN
460277,5684,20230402,1300,contributing_factor_vehicle_pair5,NaN
460278,7507,20230402,508,contributing_factor_vehicle_pair5,NaN


In [70]:
# add row about position of pair in dataframe
pairs_list = list(v_and_cf_data['variable'])
pos_list =[int(x[-1]) for x in pairs_list]
v_and_cf_data['position'] = pos_list
v_and_cf_data

,location_id,date_id,time_id,variable,value,position
0,50,20210102,1045,contributing_factor_vehicle_pair1,Driver Inattention/Distraction_STRUCKVGON/SPOR...,1
1,51,20210102,2200,contributing_factor_vehicle_pair1,Driver Inattention/Distraction_SEDANAN,1
2,52,20210102,1522,contributing_factor_vehicle_pair1,Driver Inattention/Distraction_STRUCKVGON/SPOR...,1
3,53,20210102,1745,contributing_factor_vehicle_pair1,Unspecified_SEDANAN,1
4,54,20210102,1455,contributing_factor_vehicle_pair1,Traffic Control Disregarded_SEDANAN,1
...,...,...,...,...,...,...
460275,4897,20230402,1421,contributing_factor_vehicle_pair5,NaN,5
460276,4975,20230402,40,contributing_factor_vehicle_pair5,NaN,5
460277,5684,20230402,1300,contributing_factor_vehicle_pair5,NaN,5
460278,7507,20230402,508,contributing_factor_vehicle_pair5,NaN,5


In [71]:
# drop variable row and rename value row to be contributing_factor_vehicle_pair
v_and_cf_data.rename({'value':'contributing_factor_vehicle_pair'},
             axis = 'columns',
             inplace = True)
v_and_cf_data.drop(['variable'], axis=1, inplace=True)
v_and_cf_data

,location_id,date_id,time_id,contributing_factor_vehicle_pair,position
0,50,20210102,1045,Driver Inattention/Distraction_STRUCKVGON/SPOR...,1
1,51,20210102,2200,Driver Inattention/Distraction_SEDANAN,1
2,52,20210102,1522,Driver Inattention/Distraction_STRUCKVGON/SPOR...,1
3,53,20210102,1745,Unspecified_SEDANAN,1
4,54,20210102,1455,Traffic Control Disregarded_SEDANAN,1
...,...,...,...,...,...
460275,4897,20230402,1421,NaN,5
460276,4975,20230402,40,NaN,5
460277,5684,20230402,1300,NaN,5
460278,7507,20230402,508,NaN,5


In [72]:
#Drop rows where contributing_factor_vehicle_pair is null
v_and_cf_data.dropna(inplace=True)
v_and_cf_data

,location_id,date_id,time_id,contributing_factor_vehicle_pair,position
0,50,20210102,1045,Driver Inattention/Distraction_STRUCKVGON/SPOR...,1
1,51,20210102,2200,Driver Inattention/Distraction_SEDANAN,1
2,52,20210102,1522,Driver Inattention/Distraction_STRUCKVGON/SPOR...,1
3,53,20210102,1745,Unspecified_SEDANAN,1
4,54,20210102,1455,Traffic Control Disregarded_SEDANAN,1
...,...,...,...,...,...
459842,4983,20230329,809,Unspecified_SEDANAN,5
459879,4668,20230329,1200,Unspecified_STRUCKVGON/SPORT TRUCKTILITY VEHIS...,5
459895,4447,20230329,114,Unspecified_STRUCKVGON/SPORT TRUCKTILITY VEHIS...,5
460067,7774,20230331,3,Unspecified_TAXI AND LIMOTRUCKSINEI,5


In [73]:
# Split contributing_factor_vehicle_pair into two columns
v_and_cf_data[['contributing_factor','vehicle_code']] = v_and_cf_data.contributing_factor_vehicle_pair.apply(lambda x: pd.Series(str(x).split("_")))
v_and_cf_data.drop(['contributing_factor_vehicle_pair'], axis=1, inplace=True)
v_and_cf_data

,location_id,date_id,time_id,position,contributing_factor,vehicle_code
0,50,20210102,1045,1,Driver Inattention/Distraction,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...
1,51,20210102,2200,1,Driver Inattention/Distraction,SEDANAN
2,52,20210102,1522,1,Driver Inattention/Distraction,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...
3,53,20210102,1745,1,Unspecified,SEDANAN
4,54,20210102,1455,1,Traffic Control Disregarded,SEDANAN
...,...,...,...,...,...,...
459842,4983,20230329,809,5,Unspecified,SEDANAN
459879,4668,20230329,1200,5,Unspecified,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...
459895,4447,20230329,114,5,Unspecified,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...
460067,7774,20230331,3,5,Unspecified,TAXI AND LIMOTRUCKSINEI


### Create Contributing Factor Dimension table

In [74]:
# first, copy the entire table
cf_dim = v_and_cf_data.copy()

In [75]:
# second, subset for only the wanted columns in the dimension
cf_dim = cf_dim[["contributing_factor"]]

In [76]:
# second, subset for only the wanted columns in the dimension
# third, drop duplicate rows in dimension
unique_row = ["contributing_factor"]
cf_dim = cf_dim.drop_duplicates(subset = unique_row, keep = 'first')
cf_dim = cf_dim.reset_index(drop = True)
cf_dim

,contributing_factor
0,Driver Inattention/Distraction
1,Unspecified
2,Traffic Control Disregarded
3,Oversized Vehicle
4,Unsafe Speed
5,Failure to Yield Right-of-Way
6,Passing Too Closely
7,Pedestrian/Bicyclist/Other Pedestrian Error/Co...
8,Passenger Distraction
9,Following Too Closely


In [77]:
# fourth, add a surrogate key
cf_dim.insert(0, "cf_id", range(100, 100 + len(cf_dim)))
cf_dim

,cf_id,contributing_factor
0,100,Driver Inattention/Distraction
1,101,Unspecified
2,102,Traffic Control Disregarded
3,103,Oversized Vehicle
4,104,Unsafe Speed
5,105,Failure to Yield Right-of-Way
6,106,Passing Too Closely
7,107,Pedestrian/Bicyclist/Other Pedestrian Error/Co...
8,108,Passenger Distraction
9,109,Following Too Closely


In [78]:
# fifth, add the surrogate key to the data table
v_and_cf_data = v_and_cf_data.merge(cf_dim,
                                    left_on = unique_row,
                                    right_on = unique_row,
                                    how = 'left')

v_and_cf_data.head()

,location_id,date_id,time_id,position,contributing_factor,vehicle_code,cf_id
0,50,20210102,1045,1,Driver Inattention/Distraction,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,100
1,51,20210102,2200,1,Driver Inattention/Distraction,SEDANAN,100
2,52,20210102,1522,1,Driver Inattention/Distraction,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,100
3,53,20210102,1745,1,Unspecified,SEDANAN,101
4,54,20210102,1455,1,Traffic Control Disregarded,SEDANAN,102


### Create Vehicles Dimension table


In [79]:
# first, copy the entire table
vehicles_dim = v_and_cf_data.copy()
vehicles_dim.columns


Index(['location_id', 'date_id', 'time_id', 'position', 'contributing_factor',
       'vehicle_code', 'cf_id'],
      dtype='object')

In [80]:
# second, subset for only the wanted columns in the dimension
vehicles_dim= vehicles_dim[["vehicle_code", "position"]]
# third, drop duplicate rows in dimension
unique_row = ["vehicle_code", "position"]
vehicles_dim = vehicles_dim.drop_duplicates(subset = unique_row, keep = 'first')
vehicles_dim = vehicles_dim.reset_index(drop = True)
vehicles_dim

,vehicle_code,position
0,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,1
1,SEDANAN,1
2,TRUCK TRUCKTRUCKCK,1
3,TAXI AND LIMOTRUCKSINE,1
4,"BICYCLE, TRUCKNICYCLE, or SKATEBOARDE",1
...,...,...
584,TRUCKAILEROR,5
585,TRUCKTOR TRUCKTRUCKCK GASOLINE,5
586,"E-BICYCLE, TRUCKNICYCLE, or SKATEBOARDE",5
587,ARMORED TRUCKTRUCKCK,5


In [81]:
# fourth, add a surrogate key
vehicles_dim.insert(0, "vehicle_id", range(200, 200 + len(vehicles_dim)))
vehicles_dim

,vehicle_id,vehicle_code,position
0,200,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,1
1,201,SEDANAN,1
2,202,TRUCK TRUCKTRUCKCK,1
3,203,TAXI AND LIMOTRUCKSINE,1
4,204,"BICYCLE, TRUCKNICYCLE, or SKATEBOARDE",1
...,...,...,...
584,784,TRUCKAILEROR,5
585,785,TRUCKTOR TRUCKTRUCKCK GASOLINE,5
586,786,"E-BICYCLE, TRUCKNICYCLE, or SKATEBOARDE",5
587,787,ARMORED TRUCKTRUCKCK,5


In [82]:
# fifth, add the surrogate key to the data table
v_and_cf_data = v_and_cf_data.merge(vehicles_dim,
                                    left_on = unique_row,
                                    right_on = unique_row,
                                    how = 'left')

v_and_cf_data.head()

,location_id,date_id,time_id,position,contributing_factor,vehicle_code,cf_id,vehicle_id
0,50,20210102,1045,1,Driver Inattention/Distraction,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,100,200
1,51,20210102,2200,1,Driver Inattention/Distraction,SEDANAN,100,201
2,52,20210102,1522,1,Driver Inattention/Distraction,STRUCKVGON/SPORT TRUCKTILITY VEHISWEEPER OR SN...,100,200
3,53,20210102,1745,1,Unspecified,SEDANAN,101,201
4,54,20210102,1455,1,Traffic Control Disregarded,SEDANAN,102,201


### Create Vehicles Involved Fact table


In [83]:
v_and_cf_data.columns

Index(['location_id', 'date_id', 'time_id', 'position', 'contributing_factor',
       'vehicle_code', 'cf_id', 'vehicle_id'],
      dtype='object')

In [84]:
# take a subset of fact_table for only the needed columns: which are keys and measures
vehicles_involved_fact_table = v_and_cf_data[['location_id',
                                              'date_id', 
                                              'time_id', 
                                              'vehicle_id', 
                                              'cf_id']]

vehicles_involved_fact_table

,location_id,date_id,time_id,vehicle_id,cf_id
0,50,20210102,1045,200,100
1,51,20210102,2200,201,100
2,52,20210102,1522,200,100
3,53,20210102,1745,201,101
4,54,20210102,1455,201,102
...,...,...,...,...,...
165732,4983,20230329,809,770,101
165733,4668,20230329,1200,769,101
165734,4447,20230329,114,769,101
165735,7774,20230331,3,776,101


### Step 7: Deliver Facts and Dimensions to Data Warehouse (BigQuery)

In [85]:
# create a function to load dataframes to BigQuery

def load_table_to_bigquery(df,
                          table_name,
                          dataset_id):

    dataset_id = dataset_id 

    dataset_ref = bigquery_client.dataset(dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True
    job_config.write_disposition = "WRITE_TRUNCATE"

    upload_table_name = f"{dataset_id}.{table_name}"
    
    load_job = bigquery_client.load_table_from_dataframe(df,
                                                upload_table_name,
                                                job_config = job_config)
        
    print(f"""completed loading {table_name} -- {load_job}""")

In [86]:
# load location_dim to bigquery
load_table_to_bigquery(df = location_dim,
                       table_name = "location_dim",
                       dataset_id = dataset_id)

completed loading location_dim -- LoadJob<project=fundamental-run-375818, location=US, id=49b74f59-aee1-4c20-a922-40baa6edcb3f>


In [87]:
# load weather_dim to bigquery
load_table_to_bigquery(df = weather_dim,
                       table_name = "weather_dim",
                       dataset_id = dataset_id)

/Users/naseemdabiran/opt/anaconda3/lib/python3.8/site-packages/google/cloud/bigquery/_pandas_helpers.py:603: UserWarning: Pyarrow could not determine the type of columns: temperature_category.
  warnings.warn(


completed loading weather_dim -- LoadJob<project=fundamental-run-375818, location=US, id=c334c1fd-b468-4e38-bfb6-02fa08be5301>


In [88]:
# load date_dim to bigquery
load_table_to_bigquery(df = date_dim,
                       table_name = "date_dim",
                       dataset_id = dataset_id)

completed loading date_dim -- LoadJob<project=fundamental-run-375818, location=US, id=6c91da0c-bb5a-47b9-b7f1-124801e3da0f>


In [89]:
# load time_dim to bigquery
load_table_to_bigquery(df = time_dim,
                       table_name = "time_dim",
                       dataset_id = dataset_id)

/Users/naseemdabiran/opt/anaconda3/lib/python3.8/site-packages/google/cloud/bigquery/_pandas_helpers.py:603: UserWarning: Pyarrow could not determine the type of columns: time_of_day.
  warnings.warn(


completed loading time_dim -- LoadJob<project=fundamental-run-375818, location=US, id=e2235824-f9ed-4041-8075-1cd01a9b17a4>


In [90]:
# load the fact table to bigquery as nyc_complaint_fact
load_table_to_bigquery(df = collisions_fact_table,
                       table_name = "nyc_collisions_fact",
                       dataset_id = dataset_id)

completed loading nyc_collisions_fact -- LoadJob<project=fundamental-run-375818, location=US, id=e60e75f1-680c-4676-aa0c-76719f1a27bb>


In [91]:
# load the fact table to bigquery as vehicles_involved_fact
load_table_to_bigquery(df = vehicles_involved_fact_table,
                       table_name = "vehicles_involved_fact_table",
                       dataset_id = dataset_id)

completed loading vehicles_involved_fact_table -- LoadJob<project=fundamental-run-375818, location=US, id=a712e311-3c48-4737-b9f1-ed6a22bc8027>


In [92]:
# load vehicles_dim to bigquery
load_table_to_bigquery(df = vehicles_dim,
                       table_name = "vehicles_dim",
                       dataset_id = dataset_id)

completed loading vehicles_dim -- LoadJob<project=fundamental-run-375818, location=US, id=99d9bcc3-b7a2-424a-9ca4-349a6ce11ee9>


In [93]:
# load cf_dim to bigquery
load_table_to_bigquery(df = cf_dim,
                       table_name = "contributing_factor_dim",
                       dataset_id = dataset_id)

completed loading contributing_factor_dim -- LoadJob<project=fundamental-run-375818, location=US, id=630f1789-45c4-4874-8f75-dc8fa200a431>
